## Transformando de dbz em mm/h

In [ ]:
import os
import sys
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import rasterio

def numberOfDays(self, y, m):
    leap = 0
    if y% 400 == 0:
        leap = 1
    elif y % 100 == 0:
        leap = 0
    elif y% 4 == 0:
        leap = 1
    if m==2:
        return 28 + leap
    list = [1,3,5,7,8,10,12]
    if m in list:
        return 31
    return 30
print(numberOfDays(2020, 2))

#dbz_file = sys.argv[1]
#path_out = sys.argv[2]
dbz_file = r'D:\Trabalho\faculdade\TCC\dados\climaticos\radar\dez_min\01\R13537439_201901010000.raw'
path_out = r'D:\Trabalho\faculdade\TCC\dados\climaticos\radar\dez_min\teste'

def convert_to_dbz(dbz_file, path_out):
    basename = os.path.basename(dbz_file)
    #nx = 666
    #ny = 666
    #nx = 27
    #ny = 29
    nx = 500
    ny = 500
    dbz = np.fromfile(dbz_file, dtype=np.float32).reshape(ny, nx)
    prec = np.full((ny, nx), -99.0, dtype=np.float32)

    for x in range(nx):
        for y in range(ny):
            if dbz[y, x] != -99 and dbz[y, x] <= 36:
                prec[y, x] = ((10**(dbz[y, x]/10))/200)**0.625
            elif dbz[y, x] > 36:
                prec[y, x] = ((10**(dbz[y, x]/10))/300)**0.714

    prec_file = os.path.join(path_out, basename)
    with open(prec_file, 'wb') as fn:  # Escrita do arquivo como binario
        fn.write(prec)

count_files = 0
for root, dirs, files in os.walk(directory):
    for filename in files:
        count_files+=1

 
directory = r'D:\Trabalho\faculdade\TCC\dados\climaticos\radar\dbz\dez_min'
 
# iterate over files in
# that directory
for root, dirs, files in os.walk(directory):
    for filename in tqdm(files, total = count_files):
        dbz_file = os.path.join(root, filename)
        out_dir = os.path.join(r"D:\Trabalho\faculdade\TCC\dados\climaticos\radar\mm_h", root[-2:])
        convert_to_dbz(dbz_file, out_dir)
        
        


## Transformando em acumulado horario

In [ ]:
import os
from datetime import datetime, timedelta
import sys
import glob
import numpy as np

def numberOfDays(y, m):
    leap = 0
    if y% 400 == 0:
        leap = 1
    elif y % 100 == 0:
        leap = 0
    elif y% 4 == 0:
        leap = 1
    if m==2:
        return 28 + leap
    list = [1,3,5,7,8,10,12]
    if m in list:
        return 31
    return 30
print(numberOfDays(2020, 2))

#NX = 27
#NY = 29
#NX = 37
#NY = 36

def create_hourly_acc(month):
    NX = 500
    NY = 500

    dir_input = r"D:\Trabalho\faculdade\TCC\dados\climaticos\radar\mm_h\{}".format(month)
    dir_output = r"D:\Trabalho\faculdade\TCC\dados\climaticos\radar\mm_h_acumulado\{}".format(month)
    #dir_input = sys.argv[1]
    #dir_output = sys.argv[2]
    out = open('acum_prec_hourly.csv', 'w')
    end_day = numberOfDays(2019,int(month))
    start = datetime.strptime("2019{}0101".format(month), "%Y%m%d%H") 
    end = datetime.strptime("2019{}{}23".format(month,end_day), "%Y%m%d%H")
    #start = datetime.strptime(sys.argv[3], "%Y%m%d%H")
    #end = datetime.strptime(sys.argv[4], "%Y%m%d%H")

    dict_hourly = {}
    dict_daily = {}
    datehour = start
    #acum = np.full((NY, NX), 0, dtype=np.float32)

    while datehour <= end:
        acum = np.full((NY, NX), 0, dtype=np.float32)
        pattern1 = datetime.strftime(datehour, "*%Y%m%d%H00*.*")
        pattern2 = datetime.strftime(datehour - timedelta(hours=1), "*%Y%m%d%H[!00]*.*")
        files = glob.glob(os.path.join(dir_input, pattern1)) + glob.glob(os.path.join(dir_input, pattern2))
        nfiles = len(files)
        for file in sorted(files):
            print(file)
            prec = np.fromfile(file.strip(), dtype=np.float32).reshape(NY, NX)
            np.place(prec, prec==-99, 0.0)
            np.place(prec, prec<1, 0.0)
            acum = np.add(prec, acum)

        if nfiles == 0:
            datehour = datehour + timedelta(hours=1)
            continue

        acum = acum/nfiles

        datehour_str = datetime.strftime(datehour, "%Y%m%d%H")
        date_str = datetime.strftime(datehour, "%Y%m%d")

        np.savetxt(os.path.join(dir_output, datehour_str+"00.txt"), acum, fmt='%03d')
        print(datehour_str)

    #    acum_campo = np.sum(acum)
    #    dict_hourly[datehour_str] = acum_campo
    #    if pattern1[9:11] == '00':
    #        dict_daily[date_str] = 0
    #    if not np.isnan(acum_campo):
    #        dict_daily[date_str] = dict_daily[date_str] + acum_campo

        #out.write(datehour_str+","+str(dict[datehour_str])+"\n")
        datehour = datehour + timedelta(hours=1)


    out.close()
    #plt.plot(list(dict_daily.keys()), list(dict_daily.values()))
    #plt.xticks(list(dict_daily.keys())[::2])
    #plt.xticks(rotation=90)
    #plt.show()

for month in ['01','02','03','04','05','06','07','08','09','10','11','12']:
    create_hourly_acc(month)
    


In [ ]:
import pandas as pd
import numpy as np
import os
from tqdm.notebook import tqdm
arcpy.env.addOutputsToMap = False
arcpy.env.workspace = r'D:\Trabalho\faculdade\TCC\dados\climaticos\radar\mm_h_acumulado_tifs'

def generate_rain_points_shapefile(raster, out_shp_path):
    #out_shape_path =  os.path.join(out_dir, 'data_'+ raster.split('.')[0] + '.shp')
    raster_name = os.path.basename(raster.split('.')[0])
    # Obtendo a data atraves do nome do arquivo
    year = raster_name[:4]
    month = raster_name[4:6]
    day = raster_name[6:8]
    hour = raster_name[8:10]
    minute = raster_name[10:]
    date = f'{day}/{month}/{year} {hour}:{minute}'
    #date = f'{raster_name[:4]}-{raster_name[4:6]}-{raster_name[6:8]} {raster_name[8:10]}:{raster_name[10:]}'
    
    #Cortando o raster para o extent das ruas
    raster = arcpy.management.Clip(raster, "-46.83 -24 -46.36 -23.35", "tmp_raster.tif", None, "255", "NONE", "NO_MAINTAIN_EXTENT")        
    #Mantendo apenas as celulas com chuva>=1
    raster = arcpy.ia.SetNull(raster, raster, "VALUE = 0")
    if raster.maximum>0:
        #Convertendo o raster para ponto
        arcpy.conversion.RasterToPoint(raster, out_shp_path, "Value")
        
        #Adicionando o campo da data
        arcpy.management.AddField(out_shp_path, 'data', 'DATE')

        #Preenchendo o campo de data
        with arcpy.da.UpdateCursor(out_shp_path, 'data') as cursor:
            for row in cursor:
                row[0] = date
                cursor.updateRow(row)
                
rasters = arcpy.ListRasters()

for raster in tqdm(rasters, total=len(rasters)):  
    raster_name = raster.split('.')[0]
    generate_rain_points_shapefile(raster, r'D:\Trabalho\faculdade\TCC\dados\climaticos\radar\mm_h_acumulado_shps\rain_{}.shp'.format(raster_name) )

#Faz o merge de todos os shp
#...
#Muda o nome do campo de chuva
#arcpy.management.AlterField("rain_radar_2019", "grid_code", "precipitacao", "precipitacao", "LONG", 4, "NULLABLE", "DO_NOT_CLEAR")